In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open('intents.json') as file:
    data = json.load(file)

training_sentence = []
training_labels = []
labels = []
responses = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentence.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
num_classes = len(labels)


In [3]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

# Tokenization

In [4]:
# Vectorization of Data using Tokenizer
vocab_size = 1000
embedding_dim = 64
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words = vocab_size,oov_token=oov_token)
tokenizer.fit_on_texts(training_sentence)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentence)
padded_sequences = pad_sequences(sequences,truncating='post',maxlen=max_len)


In [5]:
print(sequences)
print(padded_sequences)

[[149], [11, 10, 6], [2, 150, 26], [151], [81, 103], [66, 104], [11, 10, 152], [153], [154], [155], [105, 6], [82, 82], [105, 6, 106], [156], [9, 83, 157], [82], [27, 84, 81, 103], [158, 13, 6, 106], [159], [9, 160, 13, 161], [162], [4, 2, 5, 14, 7, 18, 58], [4, 2, 5, 14, 7, 18, 67], [4, 2, 5, 14, 7, 5, 58], [4, 2, 5, 14, 7, 5, 67], [16, 107, 6], [18, 58], [18, 67], [16, 10, 18, 58], [58], [6, 10, 85, 108], [6, 10, 85, 108, 163], [16, 107, 6], [16, 164, 6], [67], [16, 85, 6], [16, 165, 6], [14], [18, 14], [21, 6, 27, 84, 14], [4, 10, 6, 166], [4, 2, 18, 14], [4, 68, 9, 69, 6], [66, 18, 14], [4, 10, 6], [16, 10, 6], [16, 2, 167], [4, 83, 9, 168, 13], [16, 83, 9, 169, 13], [4, 10, 6], [47, 7, 3], [4, 2, 3, 47], [170, 171], [22, 10, 6, 109, 86], [4, 10, 18, 87], [87, 7, 172], [22, 2, 5, 3, 86], [3, 47], [4, 28, 3, 47], [2, 3, 86, 88, 173], [48, 110, 28, 3, 47], [4, 2, 5, 3, 87], [22, 68, 9, 174, 13, 3], [22, 68, 9, 175, 3], [4, 2, 176, 3, 89], [3, 47], [47, 3], [177, 111], [59, 111], [11,

# Training the Model

In [6]:
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim,input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16,activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))

model.compile(loss = 'sparse_categorical_crossentropy',optimizer = 'adam',metrics=['accuracy'])

model.summary()
epochs = 2000
history = model.fit(padded_sequences,np.array(training_labels),epochs=epochs)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 64)            64000     
                                                                 
 global_average_pooling1d (G  (None, 64)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                1040      
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 38)                646       
                                                                 
Total params: 65,958
Trainable params: 65,958
Non-trainable params: 0
____________________________________________________

12/12 [==============================] - 0s 5ms/step - loss: 0.0915 - accuracy: 0.9862
Epoch 151/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0898 - accuracy: 0.9890
Epoch 152/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0886 - accuracy: 0.9917
Epoch 153/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0842 - accuracy: 0.9890
Epoch 154/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0836 - accuracy: 0.9890
Epoch 155/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0817 - accuracy: 0.9890
Epoch 156/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0807 - accuracy: 0.9917
Epoch 157/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0785 - accuracy: 0.9917
Epoch 158/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0776 - accuracy: 0.9890
Epoch 159/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0750 - accuracy: 0.

12/12 [==============================] - 0s 5ms/step - loss: 0.0147 - accuracy: 0.9917
Epoch 311/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0139 - accuracy: 0.9945
Epoch 312/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0140 - accuracy: 0.9917
Epoch 313/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0141 - accuracy: 0.9917
Epoch 314/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0143 - accuracy: 0.9945
Epoch 315/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0137 - accuracy: 0.9945
Epoch 316/2000
12/12 [==============================] - 0s 3ms/step - loss: 0.0140 - accuracy: 0.9917
Epoch 317/2000
12/12 [==============================] - 0s 6ms/step - loss: 0.0134 - accuracy: 0.9945
Epoch 318/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0143 - accuracy: 0.9890
Epoch 319/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0132 - accuracy: 0.

12/12 [==============================] - 0s 5ms/step - loss: 0.0097 - accuracy: 0.9945
Epoch 471/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0092 - accuracy: 0.9945
Epoch 472/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0094 - accuracy: 0.9945
Epoch 473/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0093 - accuracy: 0.9945
Epoch 474/2000
12/12 [==============================] - 0s 6ms/step - loss: 0.0100 - accuracy: 0.9945
Epoch 475/2000
12/12 [==============================] - 0s 6ms/step - loss: 0.0092 - accuracy: 0.9945
Epoch 476/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0096 - accuracy: 0.9945
Epoch 477/2000
12/12 [==============================] - 0s 6ms/step - loss: 0.0091 - accuracy: 0.9917
Epoch 478/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0096 - accuracy: 0.9917
Epoch 479/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0099 - accuracy: 0.

12/12 [==============================] - 0s 4ms/step - loss: 0.0079 - accuracy: 0.9945
Epoch 631/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0112 - accuracy: 0.9945
Epoch 632/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0109 - accuracy: 0.9890
Epoch 633/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0091 - accuracy: 0.9917
Epoch 634/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0093 - accuracy: 0.9945
Epoch 635/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0083 - accuracy: 0.9945
Epoch 636/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0092 - accuracy: 0.9945
Epoch 637/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0090 - accuracy: 0.9917
Epoch 638/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0083 - accuracy: 0.9945
Epoch 639/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0088 - accuracy: 0.

...

12/12 [==============================] - 0s 4ms/step - loss: 0.0082 - accuracy: 0.9917
Epoch 1268/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0084 - accuracy: 0.9890
Epoch 1269/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0079 - accuracy: 0.9945
Epoch 1270/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0081 - accuracy: 0.9945
Epoch 1271/2000
12/12 [==============================] - 0s 6ms/step - loss: 0.0083 - accuracy: 0.9945
Epoch 1272/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0082 - accuracy: 0.9917
Epoch 1273/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0082 - accuracy: 0.9890
Epoch 1274/2000
12/12 [==============================] - 0s 6ms/step - loss: 0.0081 - accuracy: 0.9890
Epoch 1275/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0079 - accuracy: 0.9945
Epoch 1276/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0079 - acc

12/12 [==============================] - 0s 11ms/step - loss: 0.0079 - accuracy: 0.9945
Epoch 1426/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0083 - accuracy: 0.9917
Epoch 1427/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0094 - accuracy: 0.9945
Epoch 1428/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0080 - accuracy: 0.9945
Epoch 1429/2000
12/12 [==============================] - 0s 6ms/step - loss: 0.0081 - accuracy: 0.9945
Epoch 1430/2000
12/12 [==============================] - 0s 11ms/step - loss: 0.0080 - accuracy: 0.9945
Epoch 1431/2000
12/12 [==============================] - 0s 6ms/step - loss: 0.0081 - accuracy: 0.9890
Epoch 1432/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0079 - accuracy: 0.9945
Epoch 1433/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0078 - accuracy: 0.9945
Epoch 1434/2000
12/12 [==============================] - 0s 3ms/step - loss: 0.0079 - a

12/12 [==============================] - 0s 6ms/step - loss: 0.0086 - accuracy: 0.9917
Epoch 1584/2000
12/12 [==============================] - 0s 7ms/step - loss: 0.0082 - accuracy: 0.9945
Epoch 1585/2000
12/12 [==============================] - 0s 6ms/step - loss: 0.0082 - accuracy: 0.9945
Epoch 1586/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0080 - accuracy: 0.9917
Epoch 1587/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0078 - accuracy: 0.9945
Epoch 1588/2000
12/12 [==============================] - 0s 6ms/step - loss: 0.0080 - accuracy: 0.9917
Epoch 1589/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0080 - accuracy: 0.9917
Epoch 1590/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0081 - accuracy: 0.9917
Epoch 1591/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0079 - accuracy: 0.9917
Epoch 1592/2000
12/12 [==============================] - 0s 6ms/step - loss: 0.0079 - acc

12/12 [==============================] - 0s 5ms/step - loss: 0.0081 - accuracy: 0.9945
Epoch 1742/2000
12/12 [==============================] - 0s 6ms/step - loss: 0.0080 - accuracy: 0.9945
Epoch 1743/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0078 - accuracy: 0.9945
Epoch 1744/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0078 - accuracy: 0.9945
Epoch 1745/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0079 - accuracy: 0.9945
Epoch 1746/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0078 - accuracy: 0.9917
Epoch 1747/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0080 - accuracy: 0.9945
Epoch 1748/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0087 - accuracy: 0.9945
Epoch 1749/2000
12/12 [==============================] - 0s 6ms/step - loss: 0.0083 - accuracy: 0.9945
Epoch 1750/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0080 - acc

12/12 [==============================] - 0s 4ms/step - loss: 0.0078 - accuracy: 0.9945
Epoch 1900/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0079 - accuracy: 0.9945
Epoch 1901/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0078 - accuracy: 0.9945
Epoch 1902/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0078 - accuracy: 0.9945
Epoch 1903/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0079 - accuracy: 0.9917
Epoch 1904/2000
12/12 [==============================] - 0s 6ms/step - loss: 0.0078 - accuracy: 0.9917
Epoch 1905/2000
12/12 [==============================] - 0s 6ms/step - loss: 0.0079 - accuracy: 0.9945
Epoch 1906/2000
12/12 [==============================] - 0s 5ms/step - loss: 0.0078 - accuracy: 0.9917
Epoch 1907/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0078 - accuracy: 0.9917
Epoch 1908/2000
12/12 [==============================] - 0s 4ms/step - loss: 0.0081 - acc

## Saving the Trained Model

In [7]:
# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets
